# Basic functionality

Create a Python program that will read one “rsp” file, similar to “input.rsp”, and parse out the expected plaintexts and ciphertexts for encryption and decryption respectively using AES-128-ECB. 

The given plaintexts and ciphertexts will be hexadecimal strings.Using those inputs, test and validate the AES library you will be using, while printing the results
in the console.

For all plaintext, ciphertext pairs you can assume that the key will always be all zeros, hence you can ignore the keys while parsing an “rsp” file.

In [1]:
import re
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

ENCRYPT = 0
DECRYPT = 1

In [2]:
# Reading and displaying an .rsp file
with open("input.rsp", "r") as file:
    content = file.read()

# Display the content
from IPython.display import Markdown
Markdown(f"```\n{content}\n```")


```
[ENCRYPT]
COUNT = 0
KEY = 00000000000000000000000000000000
PLAINTEXT = f34481ec3cc627bacd5dc3fb08f273e6
CIPHERTEXT = 0336763e966d92595a567cc9ce537f5e
COUNT = 1
KEY = 00000000000000000000000000000000
PLAINTEXT = 9798c4640bad75c7c3227db910174e72
CIPHERTEXT = a9a1631bf4996954ebc093957b234589
COUNT = 2
KEY = 00000000000000000000000000000000
PLAINTEXT = 96ab5c2ff612d9dfaae8c31f30c42168
CIPHERTEXT = ff4f8391a6a40ca5b25d23bedd44a597
COUNT = 3
KEY = 00000000000000000000000000000000
PLAINTEXT = 6a118a874519e64e9963798a503f1d35
CIPHERTEXT = dc43be40be0e53712f7e2bf5ca707209
[DECRYPT]
COUNT = 0
KEY = 00000000000000000000000000000000
CIPHERTEXT = 0336763e966d92595a567cc9ce537f5e
PLAINTEXT = f34481ec3cc627bacd5dc3fb08f273e6
COUNT = 1
KEY = 00000000000000000000000000000000
CIPHERTEXT = a9a1631bf4996954ebc093957b234589
PLAINTEXT = 9798c4640bad75c7c3227db910174e72
COUNT = 2
KEY = 00000000000000000000000000000000
CIPHERTEXT = ff4f8391a6a40ca5b25d23bedd44a597
PLAINTEXT = 96ab5c2ff612d9dfaae8c31f30c42168
COUNT = 3
KEY = 00000000000000000000000000000000
CIPHERTEXT = dc43be40be0e53712f7e2bf5ca707209
PLAINTEXT = 6a118a874519e64e9963798a503f1d35


```

In [ ]:
def get_plaintext(line: str):
    plaintext = re.search(r"(?<=PLAINTEXT = )[0-9A-F]+", line, re.IGNORECASE)
    if plaintext:
        return bytes.fromhex(plaintext[0])
    else:
        raise ValueError(f"Invalid file syntax. Expected plaintext, got {line}")


def get_ciphertext(line: str):
    ciphertext = re.search(r"(?<=CIPHERTEXT = )[0-9A-F]+", line, re.IGNORECASE)
    if ciphertext:
        return bytes.fromhex(ciphertext[0])
    else:
        raise ValueError(f"Invalid file syntax. Expected ciphertext, got {line}")


def parse_file(filename: str):
    with open(filename) as rsp_file:
        lines = rsp_file.read().splitlines()

    lines = [line for line in lines if line and line[0] != '#']

    contents = {ENCRYPT: [], DECRYPT: []}
    state = -1

    i = 0

    while i < len(lines):
        line = lines[i]

        if line.upper() == "[ENCRYPT]":
            state = ENCRYPT
            i += 1
        elif line.upper() == "[DECRYPT]":
            state = DECRYPT
            i += 1
        else:
            # key = re.search(r"(?<=KEY = )\d+", lines[i+1])
            # if key:
            #     key = int(key[0])
            # else:
            #     raise ValueError(f"Invalid file syntax. Expected key, got {lines[i+1]}")

            if state == ENCRYPT:
                plaintext = get_plaintext(lines[i+2])
                ciphertext = get_ciphertext(lines[i+3])
            elif state == DECRYPT:
                ciphertext = get_ciphertext(lines[i+2])
                plaintext = get_plaintext(lines[i+3])
            else:
                raise ValueError("Report this to the developer")

            contents[state].append((b'\x00'*16, plaintext, ciphertext))
            i += 4

    return contents

In [ ]:
def encrypt(plain: bytes, key: bytes):
    cipher = Cipher(algorithms.AES128(key), modes.ECB())
    encryptor = cipher.encryptor()
    return encryptor.update(plain) + encryptor.finalize()


def decrypt(ciphertext: bytes, key: bytes):
    cipher = Cipher(algorithms.AES128(key), modes.ECB())
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()

In [ ]:
def main():
    file_contents = parse_file("input.rsp")

    print("=== Encryption ===")

    for i, test in enumerate(file_contents[ENCRYPT]):
        cipher = encrypt(test[1], test[0])
        print(f"Test {i}: target {test[2].hex()} - result {cipher.hex()} [{'PASS' if cipher==test[2] else 'FAIL'}]")

    print("=== Decryption ===")

    for i, test in enumerate(file_contents[DECRYPT]):
        plain = decrypt(test[2], test[0])
        print(f"Test {i}: target {test[1].hex()} - result {plain.hex()} [{'PASS' if plain==test[1] else 'FAIL'}]")


if __name__ == "__main__":
    main()